In [ ]:
from typing import List, Tuple

def check_complimentary(sentence1: set, sentence2: set) -> Tuple[bool, set]:
    for literal in sentence1:
        if f"~{literal}" in sentence2:
            resolved = sentence1.union(sentence2)
            resolved.discard(literal)
            resolved.discard(f"~{literal}")
            return True, resolved
        elif literal.startswith("~") and literal[1:] in sentence2:
            resolved = sentence1.union(sentence2)
            resolved.discard(literal)
            resolved.discard(literal[1:])
            return True, resolved
    return False, set()

def proof_by_resolution(Knowledge_Base: List[set], goal: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    proof_tree = []
    sentences = Knowledge_Base + [goal]
    new_sentences = []

    while True:

        pairs = [(sentences[i], sentences[j], (i, j)) for i in range(len(sentences)) for j in range(i + 1, len(sentences))]

        for sentence1, sentence2, (i, j) in pairs:
            complementary, resolved_sentence = check_complimentary(sentence1, sentence2)
            if complementary:
                if not resolved_sentence:
                    proof_tree.append((resolved_sentence, [i, j]))
                    return True, proof_tree
                if resolved_sentence not in sentences and resolved_sentence not in new_sentences:
                    new_sentences.append(resolved_sentence)
                    proof_tree.append((resolved_sentence, [i, j]))

        if not new_sentences:
            return False, proof_tree

        sentences.extend(new_sentences)
        new_sentences = []

if __name__ == "__main__":
    Knowledge_Base = [
        {"P", "Q"},       # sentence 1: P ∨ Q
        {"~P", "R"},      # sentence 2: ¬P ∨ R
        {"~R"},           # sentence13: ¬R
    ]
    goal = {"~Q"}

    success, proof_tree = proof_by_resolution(Knowledge_Base, goal)
    if success:
        print("PROVED")
    else:
        print("CAN'T BE PROVED")

    print("\nProof Tree:")
    for i, (sentence, parents) in enumerate(proof_tree):
        print(f"Step {i + 1}: {sentence} from {parents}")


PROVED

Proof Tree:
Step 1: {'R', 'Q'} from [0, 1]
Step 2: {'P'} from [0, 3]
Step 3: {'~P'} from [1, 2]
Step 4: {'Q'} from [0, 6]
Step 5: {'R'} from [1, 5]
Step 6: set() from [5, 6]


In [6]:
from itertools import combinations

def unify_sentences_var(var, x, theta):
    if var in theta:
        return unify_sentences(theta[var], x, theta)
    elif x in theta:
        return unify_sentences(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def resolve(sentence1, sentence2):
    resolvents = []
    for predicate1 in sentence1:
        for predicate2 in sentence2:
            theta = unify_sentences(predicate1, negate(predicate2))
            if theta is not None:
                new_sentence = (substitute(sentence1, theta) | substitute(sentence2, theta)) - {predicate1, predicate2}
                resolvents.append(frozenset(new_sentence))
    return resolvents

def unify_sentences(x, y, theta={}):
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_sentences_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_sentences_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify_sentences(x[1:], y[1:], unify_sentences(x[0], y[0], theta))
    else:
        return None

def negate(predicate):
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def substitute(sentence, theta):
    return {substitute_predicate(p, theta) for p in sentence}

def substitute_predicate(predicate, theta):
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])

def proof_by_resolution(Knowledge_Base, query):
    negated_query = frozenset({negate(query)})
    sentences = Knowledge_Base | {negated_query}
    new_sentences = set()

    while True:
        for sentence1, sentence2 in combinations(sentences, 2):
            resolvents = resolve(sentence1, sentence2)
            if frozenset() in resolvents:
                return True
            new_sentences.update(resolvents)
        if new_sentences.issubset(sentences):
            return False
        sentences |= new_sentences

Knowledge_Base = {
    frozenset({('Mother', 'Leela', 'Oshin')}),
    frozenset({('Alive', 'Leela')}),
    frozenset({('not','Mother', 'x','y')}),
    frozenset({('Parent','x','y')}),
    frozenset({('not','Parent', 'w', 'z')}),
    frozenset({('not','Alive','w','z')}),
    frozenset({('Older','w','z')}),
}

query = ('Older', 'Leela', 'Older')
result = proof_by_resolution(Knowledge_Base, query)
if result:
    print("Leela is older than Oshin.\nProved by resolution.")
else:
    print("Cannot prove. Leela is not older than Oshin.")

Leela is older than Oshin.
Proved by resolution.
